In [1]:

import pyspark
from delta import *

builder = pyspark.sql.SparkSession.builder.appName("DF2_Practice") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") 

spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [3]:
 pip install shapely

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 327.1 kB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [9]:
import json
from shapely.geometry import shape
from shapely.ops import unary_union

# Load GeoJSON file
with open('nyc-boroughs.geojson', 'r') as f:
    data = json.load(f)

# Initialize an empty dictionary to hold the boroughs and their corresponding geometries
boroughs = {}

# Iterate over features in the GeoJSON file
for feature in data['features']:
    # Get the borough and its geometry
    borough = feature['properties']['borough']
    geom = shape(feature['geometry'])

    # If the borough is already in the dictionary, add the new geometry to the existing one
    if borough in boroughs:
        boroughs[borough].append(geom)
    # Otherwise, add the borough to the dictionary with the geometry
    else:
        boroughs[borough] = [geom]
print(boroughs)

# Now, use unary_union to combine the geometries for each borough and calculate the area
for borough, geoms in boroughs.items():
    # Combine the geometries
    combined_geom = unary_union(geoms)
    # Calculate the area
    area = combined_geom.area
    # Replace the list of geometries in the dictionary with the area
    boroughs[borough] = area

# Now you have a dictionary where the keys are borough names and the values are their areas
print(boroughs)


{'Staten Island': [<POLYGON ((-74.051 40.566, -74.05 40.566, -74.049 40.566, -74.049 40.565, -7...>, <POLYGON ((-74.053 40.578, -74.054 40.577, -74.055 40.578, -74.055 40.58, -7...>, <POLYGON ((-74.159 40.641, -74.16 40.641, -74.161 40.642, -74.161 40.644, -7...>, <POLYGON ((-74.082 40.648, -74.081 40.649, -74.081 40.648, -74.08 40.648, -7...>], 'Queens': [<POLYGON ((-73.837 40.595, -73.837 40.595, -73.836 40.594, -73.835 40.594, -...>, <POLYGON ((-73.813 40.604, -73.812 40.603, -73.811 40.603, -73.811 40.602, -...>, <POLYGON ((-73.827 40.608, -73.827 40.608, -73.826 40.608, -73.826 40.608, -...>, <POLYGON ((-73.826 40.608, -73.827 40.608, -73.828 40.608, -73.828 40.61, -7...>, <POLYGON ((-73.832 40.606, -73.835 40.605, -73.836 40.606, -73.834 40.607, -...>, <POLYGON ((-73.794 40.608, -73.794 40.606, -73.794 40.606, -73.795 40.607, -...>, <POLYGON ((-73.805 40.597, -73.805 40.597, -73.805 40.597, -73.805 40.597, -...>, <POLYGON ((-73.805 40.597, -73.805 40.597, -73.805 40.597, -73.805 

In [10]:
# Load GeoJSON file
with open('nyc-boroughs.geojson', 'r') as f:
    data = json.load(f)

# Initialize an empty dictionary to hold the boroughs and their corresponding geometries
boroughs = {}
for feature in data['features']:
    borough = feature['properties']['borough']
    geom = shape(feature['geometry'])

    if borough in boroughs:
        boroughs[borough]['geoms'].append(geom)
    else:
        boroughs[borough] = {'geoms': [geom]}

for borough, data in boroughs.items():
    combined_geom = unary_union(data['geoms'])
    area = combined_geom.area
    boroughs[borough] = {'area': area, 'geom': combined_geom}

print(boroughs)


# Proovi kas alad klapivad ja kuidas.
# Kuidas käib punkti määramine ühte kujundisee?


{'Staten Island': {'area': 0.01604935817680098, 'geom': <MULTIPOLYGON (((-74.049 40.566, -74.049 40.565, -74.05 40.565, -74.051 40.5...>}, 'Queens': {'area': 0.03019468843503742, 'geom': <MULTIPOLYGON (((-73.804 40.598, -73.803 40.599, -73.802 40.599, -73.801 40....>}, 'Brooklyn': {'area': 0.0193814313975848, 'geom': <MULTIPOLYGON (((-73.901 40.587, -73.901 40.587, -73.901 40.588, -73.901 40....>}, 'Manhattan': {'area': 0.006306320213518681, 'geom': <MULTIPOLYGON (((-74.008 40.686, -74.011 40.684, -74.012 40.684, -74.012 40....>}, 'Bronx': {'area': 0.011778404880880338, 'geom': <MULTIPOLYGON (((-73.888 40.798, -73.887 40.798, -73.884 40.796, -73.884 40....>}}
